In [1]:
import pandas as pd

data = pd.read_csv("clean_weather.csv", index_col=0)

In [2]:
data = data.ffill()

In [195]:
train = data[["tmax", "tmin", "rain"]].to_numpy()[:-1000,:]
target = data[["tmax_tomorrow"]].to_numpy()[:-1000,:]

test = data[["tmax", "tmin", "rain"]].to_numpy()[-1000:,:]
test_target = data[["tmax_tomorrow"]].to_numpy()[-1000:,:]

In [196]:
import numpy as np

In [197]:
def mse_loss(predictions, actuals):
    return ((actuals - predictions) ** 2)

In [198]:
def finite_diff(func, x):
    eps = 1e-7
    grad = []
    for i in range(len(x)):
        row = np.zeros((1,len(x)))
        row[0,i] = 1
        grad.append((func(x + eps * row) - func(x - eps * row)) / (2 * eps))
    grad = np.array(grad).reshape(len(x), 1)
    print(grad)
    return grad

In [199]:
# Hidden

#1,3 - input
#1,10 - layer1
#1,1 - layer2

# Weights

#3,10
#10,1

#Bias

#1,10
#1,1

#Gradient
# 1,1

# Back
# Step 1
# Gradient dot activated (1,1)
    # Gradient times hidden layer 1 (1,1) * (1,10) = (1,10)
# Weights times gradient 10,1 times 1,1 T = (1,10)

#Step 2
# Gradient dot activated 1,10 dot 1,10 1,10
    # Gradient times input (1,10) * (1,3) ?
# Weights times gradient (3,10) times (10,3) (3,3)

In [200]:
class Layer():
    def __init__(self, in_shape, out_shape, relu=True):
        self.weight = np.random.rand(in_shape, out_shape)
        self.bias = np.ones((1, out_shape))
        self.relu = relu
        self.hidden = None
        self.activated = None
        self.input = None
    
    def forward(self, x):
        self.input = x
        linear = np.matmul(x, self.weight) + self.bias
        self.hidden = linear
        if self.relu:
            relu = np.maximum(linear, np.zeros(linear.shape))
            self.activated = relu
            return relu
        return linear
    
    def backward(self, gradient, prev, lr):
        hidden = self.hidden
        if self.relu:
            hidden = np.heaviside(hidden, 1)
        
        activated_grad = np.multiply(gradient, hidden).T
        
        w_grad = np.matmul(activated_grad, prev).T
        b_grad = activated_grad.T
        
        self.weight -= w_grad * lr
        self.bias -= b_grad * lr
        
        return np.matmul(self.weight, activated_grad).T

In [201]:
class Network():
    def __init__(self, layers):
        self.layers = layers
    
    def forward(self, x):
        output = np.zeros((x.shape[0],1))
        for i in range(x.shape[0]):
            vals = x[i:(i+1),:].copy()
            for layer in self.layers:
                vals = layer.forward(vals)
            output[i,:] = vals
        return output
    
    def backward(self, gradient, lr):
        for i in range(gradient.shape[0]):
            elem = gradient[i,:].reshape((1,1))
            for i in range(len(self.layers)-1, -1, -1):
                current = self.layers[i]
                if i > 0:
                    prev = self.layers[i-1].hidden
                else:
                    prev = self.layers[i].input

                elem = current.backward(elem, prev, lr)

In [202]:
layers = [
    Layer(3, 10, True),
    Layer(10, 1, True)
]

net = Network(layers)

In [203]:
epochs = 200
lr = 1e-7

for epoch in range(epochs):
    predictions = net.forward(train)
    
    if epoch % 10 == 0:
        print(f"Epoch: {epoch} MSE: {np.mean(mse_loss(predictions, target))}")
    outer_grad = predictions - target
    
    # Calc gradients
    net.backward(outer_grad, lr)

Epoch: 0 MSE: 37912.96597436931
Epoch: 25 MSE: 4400.003997122072
Epoch: 50 MSE: 4400.003997122072


KeyboardInterrupt: 

In [191]:
test_preds = net.forward(test)

In [192]:
np.mean(mse_loss(test_preds, test_target))

28.655030463939905

In [194]:
test

array([[77., 54.,  0.],
       [73., 46.,  0.],
       [77., 47.,  0.],
       ...,
       [66., 41.,  0.],
       [70., 39.,  0.],
       [62., 41.,  0.]])